# Auvix

## Инициализация

In [ ]:
import os
import sys

from django.utils import timezone

sys.path.append('/home/ubuntu/anodos.ru/anodos/')
os.environ['DJANGO_SETTINGS_MODULE'] = 'anodos.settings'

from django.core.wsgi import get_wsgi_application
application = get_wsgi_application()


import re
import catalog.runner
from catalog.models import *


class Runner(catalog.runner.Runner):

    name  = 'Auvix'
    alias = 'auvix'
    url = {
        'start'  : 'https://b2b.auvix.ru/',
        'login'  : 'https://b2b.auvix.ru/?login=yes',
        'price'  : 'https://b2b.auvix.ru/prices/Price_AUVIX_dealer_xml.zip'}

    def __init__(self):

        super().__init__()

        self.stock   = self.take_stock('stock',   'склад', 3, 10)
        self.factory = self.take_stock('factory', 'на заказ', 20, 80)

        self.count = {'product': 0, 'party': 0}

    def run(self):

        payload = {
            'AUTH_FORM'     : 'Y',
            'TYPE'          : 'AUTH',
            'backurl'       : '/',
            'USER_LOGIN'    : self.updater.login,
            'USER_PASSWORD' : self.updater.password,
            'Login'         : '%C2%A0',
            'USER_REMEMBER' : 'Y'}
        if self.login(payload):
            print('Авторизован.')
        else:
            print('Не удалось авторизоваться')
            return False

        # Загружаем данные
        self.data = self.load_data(self.url['price'])
        self.data = self.unpack_xml(self.data)
        if self.data is not None:
            print('Данные загружены.')
        else:
            print('Ошибка загрузки данных!')
            return None

s = Runner()

In [ ]:
def get_string(element, query):

    try:
        result = element.xpath(query)[0].text.strip()
    except Exception:
        result = ''

    return result

s.get_string = get_string

## Загрузка данных

In [ ]:
s.run()

## Готовим данные

In [ ]:
print(s.data)

In [ ]:
currency = {
    'USD'   : s.usd,
    'Евро'  : s.eur,
    'Рубль' : s.rub}

In [ ]:
# Перенос данных
tree = s.data

## Парсим данные

In [ ]:
s.reg = re.compile('\[(?P<article>[0-9A-Za-z\.\-\_ ]+)\]')

products = []
categories = []

for group in tree.xpath('.//Группа'):

    category = s.get_string(group, './Наименование')

    for element in group.xpath('./Товары/Товар'):

        product = {}
        party = {}

        # Производитель
        product['vendor'] = Vendor.objects.get_by_key(s.updater, s.get_string(element, './Производитель'))

        # Продукт
        product['name'] = s.fix_name(s.get_string(element, './Наименование'))

        product['article'] = s.get_string(element, './Модель')
        product['article_alt'] = re.search(s.reg, s.get_string(element, './Наименование'))
        if product['article_alt']:
            product['article'] = product['article_alt'].group('article')
        product['article'] = s.fix_article(product['article'])

        product = Product.objects.take(article  = product['article'],
                                       vendor   = product['vendor'],
                                       name     = product['name'],
                                       unit     = s.default_unit,
                                       category = category)

        if product:
            s.count['product'] += 1
        else:
            continue

        # Партии
        party['article'] = s.fix_article(s.get_string(element, './Артикул'))

        party['quantity'] = s.fix_quantity(s.get_string(element, './Количество'))

        party['currency'] = s.get_string(element, './Валюта')
        if party['currency']:
            party['currency'] = currency[party['currency']]
        else:
            party['currency'] = None

        party['price_in'] = s.fix_price(s.get_string(element, './Цена_3'))
        party['price_out'] = s.fix_price(s.get_string(element, './Цена_1'))

        if party['quantity'] != 0:
            party = Party.objects.make(product    = product,
                                       stock      = s.stock,
                                       price      = party['price_in'],
                                       price_type = s.dp,
                                       currency   = party['currency'],
                                       quantity   = party['quantity'],
                                       unit       = s.default_unit,
                                       time       = s.start_time)
            s.count['party'] += 1

        else:
            party = Party.objects.make(product    = product,
                                       stock      = s.factory,
                                       price      = party['price_in'],
                                       price_type = s.dp,
                                       currency   = party['currency'],
                                       quantity   = None,
                                       unit       = s.default_unit,
                                       time       = s.start_time)
            s.count['party'] += 1

        products.append(product)
        categories.append(category)

In [ ]:
print('Количество продуктов:', len(products))

In [ ]:
print('Количество категорий:', len(categories))

In [ ]:
for n, product in enumerate(products):
    print('{} {}'.format(n, product, categories[n]))